<a href="https://colab.research.google.com/github/Collinskib/Selenium/blob/main/Kilimall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.hjDHr3eXDs/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.Ns2ZKwGzWC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.9nLmyVhvsD/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [35]:
!pip install bs4
!pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import requests_html
from requests_html import HTMLSession, AsyncHTMLSession
from bs4 import BeautifulSoup
from time import sleep
from random import uniform

In [44]:
import selenium
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = selenium.webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
item_name = input('Enter product that you want to search for: ')
wd = selenium.webdriver.Chrome('chromedriver', options=chrome_options)
wd.get("https://www.kilimall.co.ke/new")
import time
# time.sleep(10)
search = WebDriverWait(wd, 10).until(
    EC.presence_of_element_located((By.XPATH, '//input[@class="el-input__inner"]'))
)
print('search found')
search.click()
time.sleep(5)
webdriver.ActionChains(wd).send_keys(Keys.ESCAPE).perform()
search.click()
search.clear()
search.send_keys(str(item_name))
webdriver.ActionChains(wd).send_keys(Keys.ENTER).perform()



Itemz = []
Prices = []
Ratings = []
No_of_ratings = []
Links = []

WebDriverWait(wd, 20).until(EC.presence_of_element_located((By.XPATH, "//li[@class='el-menu-item']")))
print('Search results loaded')


try:
    main = WebDriverWait(wd, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'el-row'))
    )
   
    wd.get(wd.current_url)
    item = WebDriverWait(wd, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'el-col-6'))
    )
    items = wd.find_elements(By.CSS_SELECTOR,
                              'div[class="el-col el-col-6"]')
    print('found ' + str(len(items)) + ' items')
    for item in items:
        name = item.find_element(By.XPATH,'.//div[@class="wordwrap"]/div').text
        print(name)
        price = item.find_element(By.XPATH,
                                  ".//div[@class='wordwrap-box']/div/span[1]").text
        print(price)
        ratings_el = item.find_element(By.CLASS_NAME, 'el-rate')
        rating = ratings_el.get_attribute("aria-valuenow")
        link =item.find_element(By.TAG_NAME,'a').get_attribute('href')
        print(str(link))
        print(rating)
        no_of_ratings = item.find_element(By.CLASS_NAME, 'sixtwo').text
        my_string = str(no_of_ratings)
        my_string = my_string.replace("(", "").replace(")", "")
        print(my_string)
        Itemz.append(str(name))
        Prices.append(str(price))
        Ratings.append(str(rating))
        No_of_ratings.append(my_string)
        Links.append(str(link))



except Exception as e:
    print('Got No Items because ' + str(e))




<ipython-input-44-d4201455c84b>:13: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


Enter product that you want to search for: Television
search found
Search results loaded
found 40 items
Vitron 32 Inch Frameless Smart TV HD Netflix TV Youtube Television DVB-T2 Android 11 HTC3200S AC Energy Saving Television
KSh 12,999
https://www.kilimall.co.ke/new/goods/18272411-Vitron-32-Inch-Frameless-Smart-TV-HD-Netflix-TV-Youtube-Television-DVBT2-Android-11-HTC3200S-AC-Energy-Saving-Television
4
337
VITRON HTC3200S - 32" Inch Frameless Smart Android TV Inbuilt WIFI,Bluetooth -Netflix,Youtube Television
KSh 15,299
https://www.kilimall.co.ke/new/goods/18061661-VITRON-HTC3200S--32-Inch-Frameless-Smart-Android-TV-Inbuilt-WIFIBluetooth-NetflixYoutube-Television
4
169
Vitron 32 inch Frameless Television LED Digital TV with Inbuilt Decoder DVBT2 HTC 3218
KSh 10,939
https://www.kilimall.co.ke/new/goods/18257587-Vitron-32-inch-Frameless-Television-LED-Digital-TV-with-Inbuilt-Decoder-DVBT2-HTC-3218
4
244
Vitron 4388FS,43" Inch Smart Android TV,NETFLIX,YOUTUBE,WI-FI,INBUILT DECODER
KSh 23,

In [45]:
mydictionary = {"Item": Itemz, "Price": Prices, "Rating": Ratings, 'No of Ratings': No_of_ratings, 'Link':Links}
df_shop = pd.DataFrame(mydictionary)

df_shop.head()
fname = f'Kilimall-{item_name}.csv'
df_shop.to_csv(fname)